In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
DNAME = "FoodData_Central_csv_2020-04-29/"

In [27]:
dtrain = xgb.DMatrix(DNAME+'train_reduced.csv?format=csv&label_column=1')
dtest = xgb.DMatrix(DNAME+'test_meat_target.csv?format=csv&label_column=1')

In [26]:
pd.read_csv(DNAME+"train_reduced.csv")

,fdc_id,is_meat,Nitrogen,Protein,Total lipid (fat),"Carbohydrate, by difference",Ash,Energy,Starch,Sucrose,...,22:3,20:4 c,20:5 c,22:5 c,22:6 c,20:2 c,trans-beta-Carotene,trans-Lycopene,"Cryptoxanthin, alpha",Total dietary fiber (AOAC 2011.25)
0,515053,0,0.0,5.88,29.41,64.71,0.0,529.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,374901,0,0.0,2.63,15.79,65.79,0.0,421.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,529026,0,0.0,0.00,0.00,11.67,0.0,46.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,409922,0,0.0,21.88,56.25,18.75,0.0,594.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,356996,0,0.0,2.67,0.00,8.67,0.0,47.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25099,575422,1,0.0,14.29,1.79,7.14,0.0,107.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25100,492631,1,0.0,21.18,18.82,24.71,0.0,365.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25101,504920,1,0.0,4.62,0.77,17.69,0.0,92.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25102,486534,1,0.0,11.76,12.94,25.88,0.0,271.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
"""params = {"max_depth":range(3,23,5),
          "n_estimators":[100,300,500],
          "ccp_alpha":[0,.001]
         }"""

In [14]:
params = {'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

In [28]:
hmm = xgb.train(params,dtrain)

In [29]:
ypred = hmm.predict(dtest)

In [40]:
ypred=ypred.round()
ypred

array([0., 1., 0., ..., 0., 0., 0.], dtype=float32)

In [36]:
a=pd.DataFrame(dtest.get_label())
a[0].unique()

array([0., 1.], dtype=float32)

In [41]:
from sklearn import metrics
metrics.recall_score(a, ypred)

0.9640378548895899

In [20]:
dtest.__dict__

{'missing': nan,
 'nthread': 1,
 'silent': False,
 'handle': c_void_p(140668575897024)}

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
gs = GridSearchCV(estimator=rfc, param_grid=params,scoring="recall",cv=10,verbose=1,n_jobs=-1)



In [5]:
gs.fit(x_train, y_train.values.ravel())

Fitting 10 folds for each of 24 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   41.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  8.6min finished


GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'ccp_alpha': [0, 0.001], 'max_depth': range(3, 23, 5),
                         'n_estimators': [100, 300, 500]},
             scoring='recall', verbose=1)

In [6]:
gs.best_estimator_

RandomForestClassifier(ccp_alpha=0, max_depth=18, n_estimators=300)

In [7]:
gs

GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'ccp_alpha': [0, 0.001], 'max_depth': range(3, 23, 5),
                         'n_estimators': [100, 300, 500]},
             scoring='recall', verbose=1)

In [8]:
from sklearn import metrics
gs_pred = gs.predict(x_test)
gs_recall = metrics.recall_score(y_test, gs_pred)
gs_recall

0.9883280757097792

In [10]:
gs_f1 = metrics.f1_score(y_test, gs_pred)
gs_acc = metrics.accuracy_score(y_test, gs_pred)
gs_recall = metrics.recall_score(y_test, gs_pred)
gs_prec = metrics.precision_score(y_test, gs_pred)

print('f1,recall,acc,prec score: ', gs_f1,gs_acc,gs_recall,gs_prec)

f1,recall,acc,prec score:  0.9559115179252479 0.9903295967876862 0.9883280757097792 0.9255539143279173
